## This is the new method

In [ ]:
!pip install transformers sentence-transformers faiss-cpu

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import faiss
import torch

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
with open('document1.txt', 'r') as f:
    document_text = f.read()

In [ ]:

document_chunks = document_text.split('.')

# Generate embeddings for the document chunks
document_embeddings = embedder.encode(document_chunks)

# Create a FAISS index
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
def generate_response(query, retrieved_documents):
    """Generates a response using the LLM."""
    input_ids = tokenizer(query, return_tensors="pt").input_ids
    attention_mask = tokenizer(query, return_tensors="pt").attention_mask

    #Concatenate retrieved documents with query
    input_text = query + " ".join(retrieved_documents)
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    attention_mask = tokenizer(input_text, return_tensors="pt").attention_mask

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
while True:
    query = input("Enter your query: ")
    if query.lower() == 'thank you':
        break

    # Generate an embedding for the query
    query_embedding = embedder.encode([query])[0]

    # Find the most similar documents
    k = 2  # Number of nearest neighbors
    distances, indices = index.search(query_embedding.reshape(1, -1), k)

    # Retrieve the relevant document chunks
    retrieved_documents = [document_chunks[i] for i in indices[0]]

    # Generate the response
    response = generate_response(query, retrieved_documents)

    print("Response:", response)

Enter your query: what is the document about
Response: Supervised Learning
Enter your query: explain eerything present in the document
Response: Unsupervised Learning: The model identifies patterns in data without labels Reinforcement Learning: The model learns by interacting with the environment and receiving feedback in the form of rewards or penalties.
Enter your query: I dont understand Reinforcement Learning
Response: I dont understand Reinforcement Learning Reinforcement Learning: The model learns by interacting with the environment and receiving feedback in the form of rewards or penalties Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to perform tasks without explicitly programmed instructions
Enter your query: explain reinforcement learning
Response: Reinforcement learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to perform tasks without explic